python ./trainSpeakerNet.py --eval --model ResNetSE34L --trainfunc angleproto --save_path data/test --max_frames 300 --test_list /home/joon/voxceleb/test_list.txt --test_path /home/joon/voxceleb/voxceleb1 --initial_model baseline_lite_ap.model


* --eval &rarr; evaluation mode
* -- model
* 

In [3]:
from google.colab import drive

ROOT = '/content/drive'

drive.mount(ROOT)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
!nvidia-smi

Wed Jul  8 13:03:38 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
%cd '/content/drive/My Drive/Stage-Imaging/Signal-denoising-in-the-wild/SpeakerRecognition'

/content/drive/My Drive/Stage-Imaging/Signal-denoising-in-the-wild/SpeakerRecognition


In [6]:
!pip install -q torchaudio

     |████████████████████████████████| 3.2MB 2.8MB/s 


In [7]:
import sys, time, os, argparse, socket
import numpy
import pdb
import torch
import glob
from tuneThreshold import tuneThresholdfromScore
from SpeakerNet import SpeakerNet
from DatasetLoader import DatasetLoader

In [8]:
# params
eval = True
model = 'ResNetSE34L'
trainfunc = 'angleproto'
save_path = '/content/drive/My Drive/Stage-Imaging/'
max_frames = 300
test_list = '/content/drive/My Drive/Stage-Imaging/Signal-denoising-in-the-wild/SpeakerRecognition/veri_test.txt'
test_path = '/content/drive/My Drive/Stage-Imaging/VoxCeleb1_test/wav/'
initial_model = 'baseline_lite_ap.model'

# defaults
lr_decay = .95
test_interval = 10

In [9]:
s = SpeakerNet(model = model, max_frames = max_frames)

it = 1
prevloss = float("inf")
sumloss = 0

Embedding size is 512, encoder SAP.
Initialised Pairwise Loss


In [10]:
## Load model weights
modelfiles = glob.glob('%s/model0*.model'%save_path)
modelfiles.sort()

if len(modelfiles) >= 1:
    s.loadParameters(modelfiles[-1]);
    print("Model %s loaded from previous state!"%modelfiles[-1]);
    it = int(os.path.splitext(os.path.basename(modelfiles[-1]))[0][5:]) + 1
elif(initial_model != ""):
    s.loadParameters(save_path + initial_model);
    print("Model %s loaded!"%initial_model);

for ii in range(0, it-1):
    if ii % test_interval == 0:
        clr = s.updateLearningRate(lr_decay)

Model baseline_lite_ap.model loaded!


In [11]:
if eval == True:   
  sc, lab = s.evaluateFromListSave(test_list, print_interval=100, test_path=test_path)
  result = tuneThresholdfromScore(sc, lab, [1, 0.1]);
  print('EER %2.4f'%result[1])

Reading 4700: 1.31 Hz, embed size 512
Computing 37700: 1925.20 Hz

EER 2.2322
